In [2]:
import scrapy, requests
from scrapy.http import TextResponse
from fake_useragent import UserAgent
from selenium import webdriver

In [3]:
!pip install fake_useragent scrapy-fake-useragent

     |████████████████████████████████| 1.1 MB 1.6 MB/s eta 0:00:01     |███████████████████▎            | 675 kB 1.6 MB/s eta 0:00:01
     |████████████████████████████████| 78 kB 3.9 MB/s eta 0:00:01


## 1. 프로젝트 생성

In [5]:
!scrapy startproject musinsa

New Scrapy project 'musinsa', using template directory '/home/ubuntu/.pyenv/versions/3.8.5/envs/python3/lib/python3.8/site-packages/scrapy/templates/project', created in:
    /home/ubuntu/crawling-repo-3/musinsa

You can start your first spider with:
    cd musinsa
    scrapy genspider example example.com


## 2. xpath 확인

In [7]:
UserAgent().chrome

'Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/41.0.2228.0 Safari/537.36'

In [3]:
# category links
headers = {"user-agent": UserAgent().chrome}

category_nm = '002'
main_url = "https://search.musinsa.com/category/"

req = requests.get(main_url+category_nm, headers=headers)
response = TextResponse(req.url, body=req.text, encoding='UTF-8')
response

<200 https://search.musinsa.com/category/002>

In [4]:
midsec_code = response.xpath('//*[@id="category_list"]/dl/dd/ul/li/a/@data-code').extract()
ctgr_links = list(map(response.urljoin, midsec_code))
len(ctgr_links)

20

In [5]:
midsec_code[0], ctgr_links[0]

('002022', 'https://search.musinsa.com/category/002022')

In [10]:
# item links
mid_category = ctgr_links[3]
req = requests.get(mid_category, headers=headers)
response = TextResponse(req.url, body=req.text, encoding='UTF-8')
response

<200 https://search.musinsa.com/category/002017>

In [11]:
response.url

'https://search.musinsa.com/category/002017'

In [12]:
item_links = response.xpath('//*[@id="searchList"]/li//*[@class="li_inner"]/div[1]/a/@href').extract()
len(item_links)

90

In [13]:
# item
link = item_links[66]
req = requests.get(link, headers=headers)
response = TextResponse(req.url, body=req.text, encoding='UTF-8')
response

<200 https://store.musinsa.com/app/goods/1795752>

### **selenium 사용

In [15]:
size_ls = ['S','M','L','XL', '1', '2', '3', 'SMALL', 'MEDIUM', 'LARGE', 'EXTRA LARGE']

In [17]:
title = response.xpath('/html/head/title/text()')[0].extract().split(' -')[0]
title = title.split(') ')[1].strip()
brand = response.xpath('//*[@id="product_order_info"]/div[1]/ul/li[1]/p[2]/strong/a/text()').extract()[0]
o_price = response.xpath('//*[@id="normal_price"]/text()').extract()[0]
try:
    s_price = response.xpath('//*[@id="sale_price"]/text()').extract()[0].strip()
except:
    s_price = o_price
kw = response.xpath('//*[@id="product_order_info"]/div[1]/ul/li[@class="article-tag-list"]/p/a/text()').extract()
kw = list(map(lambda s : s.replace("#",''), kw))
img_link = response.xpath('//*[@id="bigimg"]/@src').extract()[0]
img_link = response.urljoin(img_link)
link = response.url
item_id = response.url.split('/')[-1]

# -------> selenium driver 실행

options = webdriver.ChromeOptions()
options.add_argument("headless")
driver = webdriver.Chrome(options=options)
driver.get(req.url)

try:
    most_age = driver.find_element_by_xpath('//*[@id="graph_summary_area"]/strong[1]/em').text
except:
    most_age = 0
try:
    most_sex = driver.find_element_by_xpath('//*[@id="graph_summary_area"]/span').text
except:
    most_sex = 0
    
driver.find_element_by_xpath('//*[@id="option1"]/option[not(@value="")]').click()

try:
    options2_ls = driver.find_elements_by_xpath('//*[@id="option2"]/option[not(@value="")]')
    options2 = [option.get_attribute("value") for option in options2_ls]
except:
    options2 = []
driver.quit()

# <------- seleniumdriver 종료

options1 = response.xpath('//*[@id="option1"]/option[not(@value="")]/@value').extract()


# options 1과 2를 내용에 따라 color와 size에 배분

if any(item in size_ls for item in options1) == True:
    sizes = options1
    colors = options2
else:
    sizes = options2
    colors = options1

try:    
    size_kind = response.xpath('//*[@id="size_table"]/tbody/tr/th/text()').extract()

    size_details = {}
    for idx in range(len(size_kind)-1):
        sd = response.xpath(f'//*[@id="size_table"]/tbody/tr[{idx+3}]/*/text()').extract()
        size_details[sd[0]] = sd[1:]
except:
    size_details = {}
        
title, brand, o_price, s_price , kw, img_link, link, item_id, most_age, most_sex, colors, sizes, size_details, size_kind

('가먼트 워싱 빈티지 필드 재킷 [그린]',
 'PANICALE',
 '198000',
 '198000',
 [],
 'https://image.msscdn.net/images/goods_img/20210216/1795752/1795752_1_500.jpg',
 'https://store.musinsa.com/app/goods/1795752',
 '1795752',
 0,
 0,
 ['그린'],
 ['S', 'M', 'L'],
 {'S': ['70.5', '43.8', '50', '62'],
  'M': ['71.5', '45', '52', '63'],
  'L': ['72.5', '46.2', '54', '64']},
 ['MY', 'S', 'M', 'L'])

## 3. items.py

In [1]:
%%writefile musinsa/musinsa/items.py
import scrapy


class MusinsaItem(scrapy.Item):
    title = scrapy.Field()
    brand = scrapy.Field()
    o_price = scrapy.Field()
    s_price = scrapy.Field()
    kw = scrapy.Field()
    img_link = scrapy.Field()
    link = scrapy.Field()
    item_id = scrapy.Field()
    sizes = scrapy.Field()
    size_details = scrapy.Field()

Overwriting musinsa/musinsa/items.py


## 4. spider.py

- url 분석

```python
url = "https://search.musinsa.com/category/002022
        ?device=
        &d_cat_cd=002022
        &brand=
        &rate=
        &page_kind=search
        &list_kind=small
        &sort=pop
        &sub_sort=
        &page=1
        &display_cnt=90
        &sale_goods=
        &ex_soldout=
        &color=
        &price1=
        &price2=
        &exclusive_yn=
        &size=
        &tags=
        &sale_campaign_yn=
        &timesale_yn=
        &q="
```

In [2]:
%%writefile musinsa/musinsa/spiders/spider.py

import scrapy
from musinsa.items import MusinsaItem
from fake_useragent import UserAgent


class Spider(scrapy.Spider):
    name = "Musinsa"
    allow_domain = ["musinsa.com"]
    custom_settings = {
        'CONCURRENT_REQUESTS': 10,
#         'DOWNLOAD_DELAY': 2.0,
        'AUTOTHROTTLE_ENABLED': True,
        "DOWNLOADER_MIDDLEWARES": {
            "scrapy.downloadermiddlewares.useragent.UserAgentMiddleware": None,
            'scrapy.downloadermiddlewares.retry.RetryMiddleware': None,
            "scrapy_fake_useragent.middleware.RandomUserAgentMiddleware": 400,
            'scrapy_fake_useragent.middleware.RetryUserAgentMiddleware': 401,
        }
    }

    def __init__(self, midcode="001001", page="1"):
        self.start_urls = [f"https://search.musinsa.com/category/{midcode}?page={page}/"]
        super().__init__()

    def start_requests(self):
        for url in self.start_urls:
            yield scrapy.Request(url, callback=self.parse)
            
    def parse(self, response):
        links = response.xpath('//*[@id="searchList"]/li/div[contains(@class,"li_inner")]/div[2]/p[2]/a/@href').extract()
        for link in links:
            yield scrapy.Request(link, callback=self.parse_content)
            
    def parse_content(self, response):
        item = MusinsaItem()
        title = response.xpath('/html/head/title/text()')[0].extract().split(' -')[0]
        item["title"] = title.split(') ')[1]
        item["brand"] = response.xpath('//*[@id="product_order_info"]/div[1]/ul/li[1]/p[2]/strong/a/text()')[0].extract()
        try:
            item["o_price"] = response.xpath('//*[@id="goods_price"]/del/text()')[0].extract().strip()
        except:
            item["o_price"] = response.xpath('//*[@id="goods_price"]/text()')[0].extract().strip()
        try:
            item["s_price"] = response.xpath('//*[@id="sale_price"]/text()')[0].extract().strip()
        except:
            item["s_price"] = item["o_price"]
        kw = response.xpath('//*[@id="product_order_info"]/div[1]/ul/li[contains(@class, "article-tag-list")]/p/a/text()').extract()
        item["kw"] = list(map(lambda s : s.replace("#",''), kw))
        item["img_link"] = "https:" + response.xpath('//*[@id="detail_bigimg"]/div[1]/img/@src')[0].extract()
        item["link"] = response.url
        item["item_id"] = item["link"].split('/')[-1]
        size_kind = response.xpath('//*[@id="size_table"]/tbody/tr/th/text()').extract()
        item["size_details"] = {}
        for idx in range(len(size_kind)-1):
            sd = response.xpath(f'//*[@id="size_table"]/tbody/tr[{idx+3}]/*/text()').extract()
            item["size_details"][sd[0]] = sd[1:]
            
        size_category = response.xpath('//*[@id="size_table"]/thead/tr/th/text()[2]').extract()
        size_category = ','.join([item.strip() for item in size_category])
        item["size_category"] = size_category
        
        sc = response.xpath('//*[@id="size_table"]/thead/tr/th[contains(@class, "item_val")]/text()').extract()
        item["size_category"] = []
        for idx in range(len(sc)):
            if idx % 2 != 0:
                item["size_category"].append(sc[idx].strip())

        yield item


Overwriting musinsa/musinsa/spiders/spider.py


In [2]:
import requests
from fake_useragent import UserAgent
from scrapy.http import TextResponse


headers = {"user-agent": UserAgent().chrome}
url = "https://store.musinsa.com/app/goods/1582356"

req = requests.get(url, headers=headers)
response = TextResponse(req.url, body=req.text, encoding='UTF-8')
response

<200 https://store.musinsa.com/app/goods/1582356>

In [3]:
response.url

'https://store.musinsa.com/app/goods/1582356'

In [4]:
size_kind = response.xpath('//*[@id="size_table"]/tbody/tr/th/text()').extract()
size_kind

['MY', 'S', 'M', 'L', 'XL']

In [24]:
size_div = response.xpath('//*[@id="size_table"]/thead/tr/th/text()[2]').extract()
size_div = ','.join([item.strip() for item in size_div])
size_div

'총장,어깨너비,가슴단면,소매길이'

In [25]:
item = {}
for idx in range(len(size_kind)-1):
    sd = response.xpath(f'//*[@id="size_table"]/tbody/tr[{idx+3}]/*/text()').extract()
    item[sd[0]] = sd[1:]
item

{'S': ['68', '60', '61', '56'],
 'M': ['70', '61', '63', '57'],
 'L': ['72', '62', '65', '58'],
 'XL': ['74', '63', '67', '59']}

In [ ]:
        for idx in range(len(size_kind)-1):
            sd = response.xpath(f'//*[@id="size_table"]/tbody/tr[{idx+3}]/*/text()').extract()
            item["size_details"][sd[0]] = sd[1:]


In [35]:
import tree

ModuleNotFoundError: No module named 'tree'

In [36]:
!tree musinsa/ pi

/bin/bash: tree: command not found


In [ ]:
# %load musinsa/musinsa/pipelines.py

import mysql.connector
import MySQLdb


class MusinsaPipeline():

    def __init__(self):
        self.create_connection()
        self.create_table()

    def create_connection(self):
        self.conn = mysql.connector.connect(
            host='#SECRET',
            user='root',
            passwd='#SECRET',
            database='top',
            charset='utf8',
            use_unicode=True
        )
        self.curr = self.conn.cursor()

    def create_table(self):
        self.curr.execute("""DROP TABLE IF EXISTS item""")
        self.curr.execute("""create table item(
                        title text,
                        brand text,
                        o_price text,
                        s_price text,
                        img_link text,
                        link text,
                        item_id text
                        )""")

    def process_item(self, item, spider):
        self.store_db(item)
        return item

    def store_db(self, item):
        sql = ("""INSERT into item (title, brand, o_price, s_price, img_link, link, item_id)\
VALUES (%s, %s, %s, %s, %s, %s, %s)""")
        query = (
            item["title"],
            item["brand"],
            item["o_price"],
            item["s_price"],
            item["img_link"],
            item["link"],
            item["item_id"]
        )
        self.curr.execute(sql, query)
        self.conn.commit()    


In [9]:
%%writefile musinsa/musinsa/spiders/spiders.py
import scrapy
import time
from musinsa.items import MusinsaItem


class ItemSpider(scrapy.Spider):
    name = "MusinsaItem"
    allowed_domains = ["store.musinsa.com", "search.musinsa.com"]

    #     custom_settings = {
    #         "DOWNLOADER_MIDDLEWARES" : {
    #             'scrapy.downloadermiddlewares.useragent.UserAgentMiddleware': None,
    #             'scrapy.downloadermiddlewares.retry.RetryMiddleware': None,
    #             'scrapy_fake_useragent.middleware.RandomUserAgentMiddleware': 400,
    #             'scrapy_fake_useragent.middleware.RetryUserAgentMiddleware': 401,
    #         }
    #     }
    
    def __init__(self, midcode="001001", page="1"):
        self.start_urls = [f"https://search.musinsa.com/category/{midcode}?page={page}/"]
        super().__init__()

    def start_requests(self):
        for url in self.start_urls:
            yield scrapy.Request(url, callback=self.parse)
        
    def parse(self, response):
        item_links = response.xpath('//*[@id="searchList"]/li//*[@class="li_inner"]/div[1]/a/@href').extract()
        for link in item_links:
            yield scrapy.Request(link, callback=self.item_parse)
            
    def item_parse(self, response):
        item = MusinsaItem()
        
        title = response.xpath('/html/head/title/text()')[0].extract().split(' -')[0]
        item['title'] = title.split(') ')[1].strip()
        item['brand'] = response.xpath('//*[@id="product_order_info"]/div[1]/ul/li[1]/p[2]/strong/a/text()').extract()[0]
        item['o_price'] = response.xpath('//*[@id="normal_price"]/text()').extract()[0]
        try:
            item['s_price'] = response.xpath('//*[@id="sale_price"]/text()').extract()[0]
        except:
            item['s_price'] = item['o_price']
        kw = response.xpath('//*[@id="product_order_info"]/div[1]/ul/li[@class="article-tag-list"]/p/a/text()').extract()
        item['kw'] = list(map(lambda s : s.replace("#",''), kw))
        img_link = response.xpath('//*[@id="bigimg"]/@src').extract()[0]
        item['img_link'] = response.urljoin(img_link)
        item['link'] = response.url
        item['item_id'] = response.url.split('/')[-1]
        
        size_kind = []
        
        try:    
            size_kind = response.xpath('//*[@id="size_table"]/tbody/tr/th/text()').extract()

            size_details = {}
            for idx in range(len(size_kind)-1):
                sd = response.xpath(f'//*[@id="size_table"]/tbody/tr[{idx+3}]/*/text()').extract()
                size_details[sd[0]] = sd[1:]
        except:
            size_details = {}
            
        item['size_details'] = size_details
        
        yield item


Overwriting musinsa/musinsa/spiders/spiders.py


## 5. scrapy crawl

In [8]:
!pwd

/home/ubuntu/crawling-repo-3


In [3]:
%%writefile run.sh

cd /home/ubuntu/crawling-repo-3/musinsa
scrapy crawl MusinsaItem 

Overwriting run.sh


In [16]:
!ls

README.md  mss-crawler-outer.ipynb  musinsa  run.sh


In [12]:
!cd ./musinsa && ls

musinsa  outer.csv  scrapy.cfg


In [10]:
!source run.sh

2021-03-01 05:55:15 [scrapy.utils.log] INFO: Scrapy 2.4.1 started (bot: musinsa)
2021-03-01 05:55:15 [scrapy.utils.log] INFO: Versions: lxml 4.6.2.0, libxml2 2.9.10, cssselect 1.1.0, parsel 1.6.0, w3lib 1.22.0, Twisted 20.3.0, Python 3.8.5 (default, Feb 22 2021, 12:15:12) - [GCC 7.5.0], pyOpenSSL 20.0.1 (OpenSSL 1.1.1j  16 Feb 2021), cryptography 3.4.6, Platform Linux-5.4.0-1038-aws-x86_64-with-glibc2.27
2021-03-01 05:55:15 [scrapy.utils.log] DEBUG: Using reactor: twisted.internet.epollreactor.EPollReactor
2021-03-01 05:55:15 [scrapy.crawler] INFO: Overridden settings:
{'BOT_NAME': 'musinsa',
 'NEWSPIDER_MODULE': 'musinsa.spiders',
 'SPIDER_MODULES': ['musinsa.spiders']}
2021-03-01 05:55:15 [scrapy.extensions.telnet] INFO: Telnet Password: 2f3c2788a33c2bc7
2021-03-01 05:55:15 [scrapy.middleware] INFO: Enabled extensions:
['scrapy.extensions.corestats.CoreStats',
 'scrapy.extensions.telnet.TelnetConsole',
 'scrapy.extensions.memusage.MemoryUsage',
 'scrapy.extensions.logstats.LogStats']

2021-03-01 05:55:17 [scrapy.core.scraper] DEBUG: Scraped from <200 https://store.musinsa.com/app/goods/996177>
{'brand': 'MUSINSA STANDARD',
 'img_link': 'https://image.msscdn.net/images/goods_img/20190327/996177/996177_3_500.jpg',
 'item_id': '996177',
 'kw': [],
 'link': 'https://store.musinsa.com/app/goods/996177',
 'o_price': '11900',
 's_price': '11900',
 'size_details': {'L': ['74.5', '53', '58', '26'],
                  'M': ['73', '51.5', '55.5', '25'],
                  'S': ['71.5', '50', '53', '24'],
                  'XL': ['76', '54.5', '60.5', '27'],
                  'XXL': ['77.5', '56', '63', '28'],
                  'XXXL': ['79', '57.5', '65.5', '29']},
 'title': '릴렉스 핏 크루 넥 반팔 티셔츠 [화이트]'}
2021-03-01 05:55:17 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://store.musinsa.com/app/goods/1473727> (referer: https://search.musinsa.com/category/001001?page=1/)
2021-03-01 05:55:17 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://store.musinsa.com/app/goods/50169

2021-03-01 05:55:18 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://store.musinsa.com/app/goods/968977> (referer: https://search.musinsa.com/category/001001?page=1/)
2021-03-01 05:55:18 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://store.musinsa.com/app/goods/1051406> (referer: https://search.musinsa.com/category/001001?page=1/)
2021-03-01 05:55:18 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://store.musinsa.com/app/goods/750806> (referer: https://search.musinsa.com/category/001001?page=1/)
2021-03-01 05:55:18 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://store.musinsa.com/app/goods/1117534> (referer: https://search.musinsa.com/category/001001?page=1/)
2021-03-01 05:55:18 [scrapy.core.scraper] DEBUG: Scraped from <200 https://store.musinsa.com/app/goods/722159>
{'brand': 'COSTUME O’CLOCK',
 'img_link': 'https://image.msscdn.net/images/goods_img/20180222/722159/722159_2_500.jpg',
 'item_id': '722159',
 'kw': [],
 'link': 'https://store.musinsa.com/app/go

2021-03-01 05:55:19 [scrapy.core.scraper] DEBUG: Scraped from <200 https://store.musinsa.com/app/goods/1117534>
{'brand': 'MUSINSA STANDARD',
 'img_link': 'https://image.msscdn.net/images/goods_img/20190813/1117534/1117534_3_500.jpg',
 'item_id': '1117534',
 'kw': [],
 'link': 'https://store.musinsa.com/app/goods/1117534',
 'o_price': '11900',
 's_price': '11900',
 'size_details': {'L': ['74.5', '53', '58', '26'],
                  'M': ['73', '51.5', '55.5', '25'],
                  'S': ['71.5', '50', '53', '24'],
                  'XL': ['76', '54.5', '60.5', '27'],
                  'XXL': ['77.5', '56', '63', '28'],
                  'XXXL': ['79', '57.5', '65.5', '29']},
 'title': '릴렉스 핏 크루 넥 반팔 티셔츠 [크림]'}
2021-03-01 05:55:19 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://store.musinsa.com/app/goods/1290765> (referer: https://search.musinsa.com/category/001001?page=1/)
2021-03-01 05:55:19 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://store.musinsa.com/app/goods/5

2021-03-01 05:55:20 [scrapy.core.scraper] DEBUG: Scraped from <200 https://store.musinsa.com/app/goods/986761>
{'brand': 'VIVASTUDIO',
 'img_link': 'https://image.msscdn.net/images/goods_img/20190319/986761/986761_3_500.jpg',
 'item_id': '986761',
 'kw': [],
 'link': 'https://store.musinsa.com/app/goods/986761',
 'o_price': '39000',
 's_price': ' 27,300',
 'size_details': {'L': ['73.5', '55', '56.5', '23.8'],
                  'M': ['72', '53.5', '54.5', '22.8'],
                  'S': ['70.5', '52', '52.5', '21.8'],
                  'XL': ['77', '56.5', '58.5', '24.8']},
 'title': 'BOX LOGO SHORT SLEEVE JS [BLACK]'}
2021-03-01 05:55:20 [scrapy.core.scraper] DEBUG: Scraped from <200 https://store.musinsa.com/app/goods/1446291>
{'brand': 'MUSINSA STANDARD',
 'img_link': 'https://image.msscdn.net/images/goods_img/20200514/1446291/1446291_3_500.jpg',
 'item_id': '1446291',
 'kw': [],
 'link': 'https://store.musinsa.com/app/goods/1446291',
 'o_price': '18900',
 's_price': '18900',
 'size_

2021-03-01 05:55:21 [scrapy.core.scraper] DEBUG: Scraped from <200 https://store.musinsa.com/app/goods/1454812>
{'brand': 'PARTIMENTO',
 'img_link': 'https://image.msscdn.net/images/goods_img/20200520/1454812/1454812_1_500.jpg',
 'item_id': '1454812',
 'kw': [],
 'link': 'https://store.musinsa.com/app/goods/1454812',
 'o_price': '59800',
 's_price': ' 35,500',
 'size_details': {'L': ['73.5', '57', '61.5', '25.5'],
                  'M': ['71.5', '55', '59.5', '23.5']},
 'title': '[패키지][CHUBBY]EMBROIDERY TEE (5COLOR)'}
2021-03-01 05:55:21 [scrapy.core.scraper] DEBUG: Scraped from <200 https://store.musinsa.com/app/goods/969668>
{'brand': 'THISISNEVERTHAT',
 'img_link': 'https://image.msscdn.net/images/goods_img/20190228/969668/969668_2_500.jpg',
 'item_id': '969668',
 'kw': [],
 'link': 'https://store.musinsa.com/app/goods/969668',
 'o_price': '39000',
 's_price': '39000',
 'size_details': {'L': ['73.5', '51', '60', '23.8'],
                  'M': ['71', '49', '57.5', '23.1'],
         

2021-03-01 05:55:22 [scrapy.core.scraper] DEBUG: Scraped from <200 https://store.musinsa.com/app/goods/1388775>
{'brand': 'MUSINSA STANDARD',
 'img_link': 'https://image.msscdn.net/images/goods_img/20200407/1388775/1388775_1_500.jpg',
 'item_id': '1388775',
 'kw': [],
 'link': 'https://store.musinsa.com/app/goods/1388775',
 'o_price': '15900',
 's_price': '15900',
 'size_details': {'2XL': ['74', '56', '63', '28'],
                  '3XL': ['75.5', '57.5', '65.5', '29'],
                  'L': ['71', '53', '58', '26'],
                  'M': ['69.5', '51.5', '55.5', '25'],
                  'S': ['68', '50', '53', '24'],
                  'XL': ['72.5', '54.5', '60.5', '27']},
 'title': '쿨탠다드 릴렉스 핏 크루 넥 반팔 티셔츠 [블랙]'}
2021-03-01 05:55:22 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://store.musinsa.com/app/goods/1390339> (referer: https://search.musinsa.com/category/001001?page=1/)
2021-03-01 05:55:22 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://store.musinsa.com/app/goo

2021-03-01 05:55:23 [scrapy.core.scraper] DEBUG: Scraped from <200 https://store.musinsa.com/app/goods/557880>
{'brand': 'AAA',
 'img_link': 'https://image.msscdn.net/images/goods_img/20170516/557880/557880_3_500.jpg',
 'item_id': '557880',
 'kw': [],
 'link': 'https://store.musinsa.com/app/goods/557880',
 'o_price': '30000',
 's_price': ' 15,900',
 'size_details': {'L': ['80', '54', '56', '22'],
                  'M': ['75', '49', '51', '21'],
                  'S': ['68', '44', '45', '19'],
                  'XL': ['83', '59', '61', '23']},
 'title': '(2PACK'}
2021-03-01 05:55:23 [scrapy.core.scraper] DEBUG: Scraped from <200 https://store.musinsa.com/app/goods/1561528>
{'brand': 'TMAKER',
 'img_link': 'https://image.msscdn.net/images/goods_img/20200824/1561528/1561528_2_500.jpg',
 'item_id': '1561528',
 'kw': [],
 'link': 'https://store.musinsa.com/app/goods/1561528',
 'o_price': '24900',
 's_price': ' 14,940',
 'size_details': {'100': ['78', '54', '56', '24.5'],
                  '

2021-03-01 05:55:24 [scrapy.core.scraper] DEBUG: Scraped from <200 https://store.musinsa.com/app/goods/499561>
{'brand': 'CHAMPION',
 'img_link': 'https://image.msscdn.net/images/goods_img/20170315/499561/499561_8_500.jpg',
 'item_id': '499561',
 'kw': [],
 'link': 'https://store.musinsa.com/app/goods/499561',
 'o_price': '50000',
 's_price': ' 18,900',
 'size_details': {},
 'title': '(2PACK'}
2021-03-01 05:55:24 [scrapy.core.scraper] DEBUG: Scraped from <200 https://store.musinsa.com/app/goods/996498>
{'brand': 'MUSINSA STANDARD',
 'img_link': 'https://image.msscdn.net/images/goods_img/20190327/996498/996498_3_500.jpg',
 'item_id': '996498',
 'kw': [],
 'link': 'https://store.musinsa.com/app/goods/996498',
 'o_price': '10900',
 's_price': '10900',
 'size_details': {'L': ['69.5', '46', '54', '24'],
                  'M': ['68', '44.5', '51.5', '23'],
                  'S': ['66.5', '43', '49', '22'],
                  'XL': ['71', '47.5', '56.5', '25'],
                  'XXL': ['72.5'

In [1]:
import pandas as pd

df = pd.read_csv('./musinsa/outer.csv')
df.head()

/home/ubuntu/.pyenv/versions/3.8.5/envs/python3/lib/python3.8/site-packages/pandas/compat/__init__.py:97: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)


,brand,colors,img_link,item_id,kw,link,most_age,most_sex,o_price,s_price,size_details,sizes,title
0,PARTIMENTO,NaN,https://image.msscdn.net/images/goods_img/2019...,1204448,"롱코트,야상,코트,3M신슐레이트,카키,신슐레이트패딩,신슐레이트,연말빅세일",https://store.musinsa.com/app/goods/1204448,19~23,남성,155000,"99,900","{'M': ['97', '59', '67', '63'], 'L': ['99', '6...","M,L",3M 신슐레이트 M-51 피시테일 코트 패딩 카키
1,MELANGE MASTER,NaN,https://image.msscdn.net/images/goods_img/2017...,640839,"무지스웨트,세트업,스웨트,스웨트셔츠,스웻,스탠다드,후드집업,단독후디,일석이조룩,스쿨룩",https://store.musinsa.com/app/goods/640839,19~23,남성,39900,39900,"{'S': ['65', '46.5', '52', '61'], 'M': ['67', ...","S,M,L,XL,XXL",스탠다드 후드 스웨트 집업
2,ELLIOTI,NaN,https://image.msscdn.net/images/goods_img/2021...,1784744,NaN,https://store.musinsa.com/app/goods/1784744,0,0,89000,"80,100","{'M': ['69.5', '52', '62', '61'], 'L': ['71', ...","M,L",스탠다드 블루종_베이지
3,ELLIOTI,NaN,https://image.msscdn.net/images/goods_img/2021...,1785840,NaN,https://store.musinsa.com/app/goods/1785840,0,0,89000,"80,100","{'M': ['69.5', '52', '62', '61'], 'L': ['71', ...","M,L",스탠다드 블루종_페이즐리
4,ELLIOTI,NaN,https://image.msscdn.net/images/goods_img/2021...,1785843,NaN,https://store.musinsa.com/app/goods/1785843,0,0,89000,"80,100","{'M': ['69.5', '52', '62', '61'], 'L': ['71', ...","M,L",스탠다드 블루종_블랙


In [10]:
df.columns

Index(['brand', 'colors', 'img_link', 'item_id', 'kw', 'link', 'most_age',
       'most_sex', 'o_price', 's_price', 'size_details', 'sizes', 'title'],
      dtype='object')

In [11]:
df.colors.unique()

array([nan, 'OS,OS+팬시 후원 다꾸팩|8500',
       'XS(90),S(95),M(100),L(105),XL(110),2XL(115),3XL(120)', 'colors',
       '블랙(BK)', '브라운(BR)', '검정,라벤더,아이보리,연베이지,연카키,차콜,청회색', '그레이', '베이지',
       '파란색', '(19)BLACK,(49)KHAKI', 'BLACK,LIGHT GREY,ZET BLACK',
       '095,100,105', '차콜', '검정', '95,100,105', '그린', 'INDIGO',
       '95,100,105,110', 'GREY(35),BLACK(39),DK-GREEN(68)',
       'WASHED BLACK', 'BK,LGR', '짙은회색', '아이보리', 'purple', '하늘색',
       '(35)BEIGE', 'white', '(19)BLACK,(35)BEIGE', 'black', 'GN3(다크그린)',
       'GREY', 'LAVENDER', 'WHITE', 'BLUE', 'Ivory,Black', 'Brown,Black',
       'NONE', 'BEIGE HERRINGBONE', '블랙', 'Black', 'Pink', 'LIGHT BEIGE',
       'ETOUPE', 'BEIGE', 'BEIGE,BLACK', 'BK2(블랙)', '카키', 'BLK0_BLK',
       'S,M,L', 'BLACK', '그레이.INY610438', '그레이.A0Z635136', '인디고', '블루',
       'INDIGO(SELVAGE)', '네이비', '라이트블루', 'beige', 'T32B1NJC63TWT11AB',
       '연청', 'D NAVY', 'L BEIGE', 'NAVY/BROWN', 'BROWN CHECK',
       'BROWN HERRINGBONE', 'KAKHI GREY', 'T12B1WOT130MT1AEH',

In [2]:
df.head(10)

,brand,colors,img_link,item_id,kw,link,most_age,most_sex,o_price,s_price,size_details,sizes,title
0,PARTIMENTO,NaN,https://image.msscdn.net/images/goods_img/2019...,1204448,"롱코트,야상,코트,3M신슐레이트,카키,신슐레이트패딩,신슐레이트,연말빅세일",https://store.musinsa.com/app/goods/1204448,19~23,남성,155000,"99,900","{'M': ['97', '59', '67', '63'], 'L': ['99', '6...","M,L",3M 신슐레이트 M-51 피시테일 코트 패딩 카키
1,MELANGE MASTER,NaN,https://image.msscdn.net/images/goods_img/2017...,640839,"무지스웨트,세트업,스웨트,스웨트셔츠,스웻,스탠다드,후드집업,단독후디,일석이조룩,스쿨룩",https://store.musinsa.com/app/goods/640839,19~23,남성,39900,39900,"{'S': ['65', '46.5', '52', '61'], 'M': ['67', ...","S,M,L,XL,XXL",스탠다드 후드 스웨트 집업
2,ELLIOTI,NaN,https://image.msscdn.net/images/goods_img/2021...,1784744,NaN,https://store.musinsa.com/app/goods/1784744,0,0,89000,"80,100","{'M': ['69.5', '52', '62', '61'], 'L': ['71', ...","M,L",스탠다드 블루종_베이지
3,ELLIOTI,NaN,https://image.msscdn.net/images/goods_img/2021...,1785840,NaN,https://store.musinsa.com/app/goods/1785840,0,0,89000,"80,100","{'M': ['69.5', '52', '62', '61'], 'L': ['71', ...","M,L",스탠다드 블루종_페이즐리
4,ELLIOTI,NaN,https://image.msscdn.net/images/goods_img/2021...,1785843,NaN,https://store.musinsa.com/app/goods/1785843,0,0,89000,"80,100","{'M': ['69.5', '52', '62', '61'], 'L': ['71', ...","M,L",스탠다드 블루종_블랙
5,ELLIOTI,NaN,https://image.msscdn.net/images/goods_img/2021...,1785842,NaN,https://store.musinsa.com/app/goods/1785842,0,0,89000,"80,100","{'M': ['69.5', '52', '62', '61'], 'L': ['71', ...","M,L",스탠다드 블루종_체크
6,ROTHCO,NaN,https://image.msscdn.net/images/goods_img/2014...,128887,408건,https://store.musinsa.com/app/goods/128887,19~23,남성,79000,"63,200",{},"S,M|8000,L|8000,XL|8000,2XL(오버핏)|12000,4XL(오버핏...",MA-1 FLIGHT JACKET (SAGE GREEN)
7,ITSABOUTDETAIL,NaN,https://image.msscdn.net/images/goods_img/2021...,1794932,NaN,https://store.musinsa.com/app/goods/1794932,0,0,99000,"72,000","{'M': ['70', '45', '43', '59'], 'L': ['74', '4...","M,L",3계절용 Allround Jumper
8,NASTY FANCY CLUB,"OS,OS+팬시 후원 다꾸팩|8500",https://image.msscdn.net/images/goods_img/2021...,1777819,NaN,https://store.musinsa.com/app/goods/1777819,0,0,85000,85000,"{'OS': ['67', '51', '63', '62']}",NaN,[NF] 피그먼트 윈드브레이커 자켓 (챠콜)_F611
9,261EUI,NaN,https://image.msscdn.net/images/goods_img/2021...,1791105,NaN,https://store.musinsa.com/app/goods/1791105,0,0,198000,"168,300","{'1': ['66', '62', '64', '58'], '2': ['68', '6...","1,2",SIGNATURE WRINKLE MA-1 WHITE


In [ ]:
class Mammal(object):
  def __init__(self, mammalName):
    print(mammalName, 'is a warm-blooded animal.')
    
class Dog(Mammal):
  def __init__(self):
    print('Dog has four legs.')
#     super().__init__('Dog')
    
d1 = Dog()